In [1]:
import os

from snowflake.snowpark import Session
import pandas as pd
import weaviate

## Create Weaviate Schema

Create Snowpark session through a combination of automatic environment variables and those explicitly specified in the spec yaml file.

In [2]:
def get_token():
    with open('/snowflake/session/token', 'r') as f:
        return f.read()

connection_params = {
    'host': os.environ['SNOWFLAKE_HOST'], # Automatically created in SPCS
    'port': os.environ['SNOWFLAKE_PORT'], # Automatically created in SPCS
    'protocol': 'https',
    'account': os.environ['SNOWFLAKE_ACCOUNT'], # Automatically created in SPCS
    'authenticator': 'oauth',
    'token': get_token(),
    'role': os.environ['SNOW_ROLE'], # Explicitly created in spec file's env section
    'warehouse': os.environ['SNOW_WAREHOUSE'], # Explicitly created in spec file's env section
    'database': os.environ["SNOW_DATABASE"], # Explicitly created in spec file's env section
    'schema': os.environ["SNOW_SCHEMA"] # Explicitly created in spec file's env section
}


session = Session.builder.configs(connection_params).create()

Establish a connection to the running Weaviate service through its endpoint. The final line in the below cell indicates that the service is operational.

In [3]:
client = weaviate.Client(
    url = os.environ["WEAVIATE_URL"],
)
client.is_ready()

True

In [4]:
def get_properties(client) -> list[str]:
    """Returns all properties of first weaviate class"""
    properties = client.schema.get()['classes'][0]['properties']
    names = [i.get('name') for i in properties]
    return names

weaviate_properties = get_properties(client)

RAG from a chat agent is conducted using a `retriever`. Retrievers in langchain are source-specific, meaning retrieval from our weaviate service requires a weaviate-specific retriever from langchain. Langchain offers a number of `retriever` classes. The `WeaviateHybridSearchRetriever` is one of the more advanced out of the box `retrievers` as it combines keyword search and semantic meaning. Recall that our weaviate currently stores most text-based metadata and the vectorized form of our generated descriptions.

Most retrievers allow the ability to parameterize the inclusion of `scores`. Unfortunately, as of December 2023, this is not possible in langchain so the below custom class overrides `WeaviateHybridSearchRetriever` to include `scores` by default. Doing this is entirely optional and done exclusively for testing to determine if there is a desire to set a threshold for `scores`. The current Streamlit UI simply uses `WeaviateHybridSearchRetriever` out of the box. See the below cells for an explanation of parameters. More info at https://python.langchain.com/docs/integrations/retrievers/weaviate-hybrid.

In [5]:
# https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/retrievers/weaviate_hybrid_search.py#L12
from __future__ import annotations

from typing import Any, Dict, List, Optional, cast
from uuid import uuid4

from langchain_core.pydantic_v1 import root_validator
from langchain_core.retrievers import BaseRetriever

from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.docstore.document import Document

from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever

class MyWeaviateHybridSearchRetriever(WeaviateHybridSearchRetriever):
    def _get_relevant_documents(
        self,
        query: str,
        *,
        run_manager: CallbackManagerForRetrieverRun,
        where_filter: Optional[Dict[str, object]] = None,
        score: bool = True, # Set to default to True
        hybrid_search_kwargs: Optional[Dict[str, object]] = None,
    ) -> List[Document]:
        """Look up similar documents in Weaviate.

        query: The query to search for relevant documents
         of using weviate hybrid search.

        where_filter: A filter to apply to the query.
            https://weaviate.io/developers/weaviate/guides/querying/#filtering

        score: Whether to include the score, and score explanation
            in the returned Documents meta_data.

        hybrid_search_kwargs: Used to pass additional arguments
         to the .with_hybrid() method.
            The primary uses cases for this are:
            1)  Search specific properties only -
                specify which properties to be used during hybrid search portion.
                Note: this is not the same as the (self.attributes) to be returned.
                Example - hybrid_search_kwargs={"properties": ["question", "answer"]}
            https://weaviate.io/developers/weaviate/search/hybrid#selected-properties-only

            2) Weight boosted searched properties -
                Boost the weight of certain properties during the hybrid search portion.
                Example - hybrid_search_kwargs={"properties": ["question^2", "answer"]}
            https://weaviate.io/developers/weaviate/search/hybrid#weight-boost-searched-properties

            3) Search with a custom vector - Define a different vector
                to be used during the hybrid search portion.
                Example - hybrid_search_kwargs={"vector": [0.1, 0.2, 0.3, ...]}
            https://weaviate.io/developers/weaviate/search/hybrid#with-a-custom-vector

            4) Use Fusion ranking method
                Example - from weaviate.gql.get import HybridFusion
                hybrid_search_kwargs={"fusion": fusion_type=HybridFusion.RELATIVE_SCORE}
            https://weaviate.io/developers/weaviate/search/hybrid#fusion-ranking-method
        """
        query_obj = self.client.query.get(self.index_name, self.attributes)
        if where_filter:
            query_obj = query_obj.with_where(where_filter)

        if score:
            query_obj = query_obj.with_additional(["score"]) # Removed explain score

        if hybrid_search_kwargs is None:
            hybrid_search_kwargs = {}

        result = (
            query_obj.with_hybrid(query, alpha=self.alpha, **hybrid_search_kwargs)
            .with_limit(self.k)
            .do()
        )
        if "errors" in result:
            raise ValueError(f"Error during query: {result['errors']}")

        docs = []

        for res in result["data"]["Get"][self.index_name]:
            text = res.pop(self.text_key)
            docs.append(Document(page_content=text, metadata=res))
        return docs
    
myretriever = MyWeaviateHybridSearchRetriever(
    client=client,
    index_name=os.environ["INDEX_NAME"], # Class name in Weaviate schema
    text_key=os.environ["TEXT_KEY"], # Page content in query results
    attributes = weaviate_properties, # What properties to return
    properties= weaviate_properties, # Limits the set of properties that will be searched by the BM25 component of the search
    alpha = .8, # Balances keyword search with semantic embedding search (0.5 means equal weighting)
    k = 10 # Number of documents to return
)

Below we create the first of 2 conversational chat agents. We start with a purely retrieval-based conversational agent meaning it is simply instructed to answer questions based on context (served up by our `retriever`), the question asked, and the conversation history. This is the chat agent currently used in the Streamlit UI.

The LLM inferencing is done through the `vllm` API, which is running in its own containerized service. The beauty of VLLM is that it not only serves up an API with one line of code but allows us to wrap that API in an OpenAI-like chat interface. We pass a service endpoint URL and the model namethe to the chat mechanism. The `MODEL_NAME` (environment variable) is set to `openchat35` in both `chat.yaml` and `vllm/vllm.yaml`. This `MODEL_NAME` is set for the API when the container executes `vllm/entrypoint.sh`. If using a chat mechanism directly from openai python package you may need to set the `MODEL_NAME` to an existing OpenAI model name such as `gpt-3.5-turbo`. Some of the openai python modules conduct validation to confirm the model name correspond to official OpenAI model. Fortunately, VLLM makes it easy to mitigate this validation concern.

Lastly, we can use `ConversationalRetrievalChain`'s default prompt instructions. However, we can also customize the prompt commonly done to hone style, instructions, or format. To do so, we have includes a `system_prompt.yaml`, which uses langchain's prompt serialization (https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/prompt_serialization) to import it from a yaml file. Results are not streamed in this demo but are streamed in the streamlit app. Results here can take ~30 seconds to return but are very fast streaming in the streamlit app. 

In [6]:
def prep_dns(text):
    """Makes DNS/API URL all lowercase and replaces _ with -."""
    return text.lower().replace("_", "-")

In [11]:
# Create high level agent constructor.
from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory # Can use ConversationBufferWindowMemory(k=5) to specify sliding window 
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
import openai

model = os.environ["MODEL_NAME"]
api_base = prep_dns(f'http://api.{os.environ["SNOW_SCHEMA"]}.{os.environ["SNOW_DATABASE"]}.snowflakecomputing.internal:8000/v1')

llm = ChatOpenAI(
    model=model,
    openai_api_key="EMPTY",
    openai_api_base=api_base,
    max_tokens=600,
    temperature=0.5
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key='answer')

# Create the chat prompt templates
system_message_prompt = SystemMessagePromptTemplate(prompt=load_prompt('system_prompt.yaml'))
messages = [
    system_message_prompt,
    HumanMessagePromptTemplate.from_template("{question}")
]
qa_prompt = ChatPromptTemplate.from_messages(messages)

chain = ConversationalRetrievalChain.from_llm(llm, 
                                              myretriever, # My version with score = True
                                              return_source_documents=True, # Returns results from weaviate in response
                                              memory = memory,
                                              verbose = False,
                                              combine_docs_chain_kwargs={"prompt": qa_prompt}, # Use customized prompt
                                              get_chat_history=lambda h : h) # Resolves bug in langchain chat history mechanism

In [8]:
print(chain.combine_docs_chain)
print('')
print(chain.question_generator.prompt)

llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['chat_history', 'context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'context'], template="You are a customer service agent with knowledge of rewards/offers for product purchases.\nSummarise the relevant offers below to answer the user's question only if it's relevant.\nIf you cannot find the answer from the context or chat history, \njust say that you don't know, don't make up an answer. \nDo not make up companies, products, rewards, or programs. Be concise.\nWhen describing offers, always include the company name, product name, and URL/hyperlink\nif included in the context. \nOnly reference conversation history if asked to do so, otherwise rely on the new context.\nWhen you are listing multiple offers from multiple companies, you MUST use the format:\nCompany A:\n    - offer 1\n    - offer 2\nCompany B]:\n    - offer 1\n    - offer 2\n----------------\nRelevan

In [9]:
def unpack_full_result(result):
    """Helper function to view results in more readable manner. """
    print(f"QUESTION: {result['question']}")
    print(f"\nANSWER: {result['answer']}")
    source_docs = result['source_documents']
    print('\nRAG RESULTS:')
    for i in source_docs:
        print(i.page_content)
        for k,v in i.metadata.items():
            print(f' {k} : {v}')

In [12]:
query = "Are there any rewards for pet supplies"
result = chain({"question": query})
unpack_full_result(result)

QUESTION: Are there any rewards for pet supplies

ANSWER:  Yes, there are rewards for pet supplies. Here are the relevant offers:

Company Pet Parents:
- $10 cash back for Pets category product: Purchase Pet Toy Set and write a review on the website. More info at www.petparents.com/pettoyset.
- $10 cash back for Pet Supplies category product: Purchase Cat Toy Set and share a photo of your cat playing with the toys on social media with #PetLovers. More info at www.petlovers.com/cattoyset.

Company Pet Lovers:
- $25 gift card for Pet Supplies category product: Purchase Pet Bed and write a review on the website. More info at www.petlovers.com/petbed.
- $25 cash back for Pets category product: Purchase Automatic Pet Feeder and write a review on the website. More info at www.petlovers.com/automaticpetfeeder.
- $15 cash back for Pets category product: Purchase Pet Toy Bundle and write a review on the website. More info at www.petlovers.com/toybundle.

Company Pet Owners:
- $30 gift card for 

In [13]:
query = "Show me 5 rewards for outdoor equipment."
result = chain({"question": query})
print(result['answer'])

 Company Adventure Seekers:
- $5 gift card for Outdoor Gear category product: Purchase Outdoor Backpack and write a review on our website. More info at www.adventureseekers.com/outdoorbackpack.
- $50 cash back for Outdoor & Recreation category product: Purchase Outdoor Backpack and write a review on the website. More info at www.adventureseekers.com/outdoorbackpack.
- $50 discount code for Sports & Outdoors category product: Purchase Outdoor Camping Gear and enter code ADVENTURE50 at checkout. More info at www.adventureseekers.com/outdoorfurniture.

Company Outdoor Adventurers:
- $0 discount code for Outdoor Recreation category product: Purchase Camping Gear Set and enter code OUTDOORFUN at checkout. More info at www.outdooradventurers.com/campinggearset.

Company Adventure Seekers:
- $50 discount code for Outdoor & Adventure category product: Purchase Outdoor Camping Gear and enter code ADVENTURE50 at checkout. More info at www.adventureseekers.com/outdoorcampinggear.

Company Adventu

## Using Chat Agent with Tool(s)

`ConversationalRetrievalChain` is purpose built for RAG. We can move to a purely conversational agent with access to multiple tools if we want to allow the agent to do more than retrieval. To do so, we use `initialize_agent` instead of `ConversationalRetrievalChain`. RAG is passed as one of multiple tools and handled by `RetrievalQA.from_chain_type`. The agent determines which tool(s) to use based on the tool description. We have also customized the prompts for additional clarity.

In [14]:
from langchain.chains import RetrievalQA
from langchain.agents import AgentType, Tool, initialize_agent
from langchain import LLMMathChain
from langchain.load.dump import dumps

Default prompts at https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/conversational/prompt.py

In [15]:
PREFIX = """Assistant is a large language model.

Assistant is designed to be able to assist users in finding available product reward offers for common purchases. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions about product rewards and offers. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on rewards and offers.

Assistant is designed to format relevant offers in a readable manner by grouping them according to the company providing the offer. Assistant likes to use numbered and bulleted lists when describing multiple offers.

TOOLS:
------

Assistant has access to the following tools:"""

In [16]:
FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response]
```"""

In [17]:
SUFFIX = """Only reference conversation history if asked to do so. Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}"""

In [22]:
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=myretriever
)

# calculator = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Product Rewards",
        func=ruff.run,
        description="Useful for when you need to answer questions about products and rewards. Input should be a fully formed question.",
        return_direct=True,
    )
    # ,
    # Tool(
    #     name = "Calculator",
    #     func=calculator.run,
    #     description = "Useful when you need to do math operations or arithmetic."
    #         )
]

memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", output_key="output", return_messages=True)

agent = initialize_agent(
    tools, 
    llm, 
    handle_parsing_errors=True, # Minor parsing errors can be resolved by the LLM itself
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True, # Will print thought process of LLM
    memory = memory,
    return_intermediate_steps=True, # Great for understanding how the LLM decides what to do when
    agent_kwargs={
        'prefix':PREFIX,
        'format_instructions':FORMAT_INSTRUCTIONS,
        'suffix':SUFFIX
    }
)

In [23]:
print(agent.agent.llm_chain.prompt.template)

Assistant is a large language model.

Assistant is designed to be able to assist users in finding available product reward offers for common purchases. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions about product rewards and offers. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on rewards and offers.

Assistant is designed to format relevant offers in a readable manner by grouping them according to the company providing the offer. Ass

In [24]:
response = agent({"input": "What rewards are there for cat toys?"})
print(dumps(response["intermediate_steps"], pretty=True))



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? Yes
Action: Product Rewards
Action Input: What rewards are there for cat toys?
Observation:  There are several rewards for cat toys offered by different companies:

1. Company Pet Lovers: $10 cash back for a Pet Supplies category product. To receive the reward, purchase a Cat Toy Set and share a photo of your cat playing with the toys on social media with #PetLovers.
2. Company Pet Parents: $10 cash back for a Pets category product. To receive the reward, purchase a Pet Toy Set and write a review on their website.
3. Company Pet Care: $5 cash back for a Pets category product. To receive the reward, purchase a Cat Tree and write a review on their website.
4. Company Pet Paradise: $15 cash back for a Pet Supplies category product. To receive the reward, purchase a Cat Tree and write a review on their website.
5. Company Pet Owners: $25 gift card for a Pet Supplies category product. To receive the reward, purchase 

In [25]:
response = agent({"input": "Can you show me 5 offers related to outdoor equipment?"})
print(response['output'])
print('')
print(dumps(response["intermediate_steps"], pretty=True))



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? Yes
Action: Product Rewards
Action Input: 5 offers related to outdoor equipment
Observation:  Here are five offers related to outdoor equipment:

1. Company Outdoor Adventurers: Get a $0 discount on the Outdoor Recreation category product, Camping Gear Set, by using the code OUTDOORFUN at checkout. More info at www.outdooradventurers.com/campinggearset.
2. Company Outdoor Enthusiasts: Receive a $40 discount on the Outdoor & Adventure category product, Camping Gear Bundle, by using the code OUTDOOR40OFF at checkout. More info at www.outdoorenthusiasts.com/campinggear.
3. Company Adventure Seekers: Get a $5 gift card for the Outdoor Gear category product, Outdoor Backpack, by purchasing it and writing a review on their website. More info at www.adventureseekers.com/outdoorbackpack.
4. Company Adventure Seekers: Receive a $50 cash back for the Outdoor & Recreation category product, Outdoor Backpack, by purchasing i

In [26]:
response = agent({"input": "What was the first thing I asked you?"})
print(response['output'])
print('')
print(dumps(response["intermediate_steps"], pretty=True))



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? No
AI: The first thing you asked me was, "What rewards are there for cat toys?"

> Finished chain.
The first thing you asked me was, "What rewards are there for cat toys?"

[]


## Using RAG Source Docs to Initiate Snowpark Query

Next, we show how the `source_documents` results from RAG can be used to initiate Snowpark queries. This is an important capability cause it allows us to take advantage of Snowflake RBAC by restricting queries to only those permitted by the end user. For example, we can use the RAG results to query a table of recommendation scores for the given user. This brings another element of personalization that can be added to the UI. See Streamlit for function to extract logged in user.

In [27]:
# Re-create the chain to start with fresh results
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key='answer')

chain = ConversationalRetrievalChain.from_llm(llm, 
                                              myretriever, # My version with score = True
                                              return_source_documents=True, # Returns results from weaviate in response
                                              memory = memory,
                                              verbose = False,
                                              combine_docs_chain_kwargs={"prompt": qa_prompt}, # Use customized prompt
                                              get_chat_history=lambda h : h) # Resolves bug in langchain chat history mechanism

In [28]:
query = "Are there any rewards for health & wellness products"
result = chain({"question": query})
print(result['answer'])

 Yes, there are rewards for health & wellness products. Here are some relevant offers:

1. Company Wellness Warriors:
    - Offer: $20 gift card
    - Product: Health & Wellness category product (Organic Superfood Powder)
    - Requirement: Purchase the product and share a photo of your healthy recipe on social media with #WellnessWarriors
    - Link: www.wellnesswarriors.com/superfoodpowder

2. Company Health & Wellness:
    - Offer: $25 cash back
    - Product: Health & Nutrition category product (Organic Superfood Blend)
    - Requirement: Purchase the product and share a photo of your nutritious meal on social media with #HealthWellness
    - Link: www.healthwellness.com/superfoodblend

3. Company Wellness Enthusiasts:
    - Offer: $60 cash back
    - Product: Wellness Products category product (Essential Oil Set)
    - Requirement: Purchase the product and share a photo of your self-care routine on social media with #WellnessEnthusiasts
    - Link: www.wellnessenthusiasts.com/esse

In [30]:
def get_property_value(result, property_name):
    """Returns list of property values from result metadata"""
    values = []
    if 'source_documents' in result:
        for i in result['source_documents']:
            values.append(i.metadata.get(property_name))
        return values

product_table = os.environ["PRODUCT_TABLE"] # Snowflake table to query
# Get the unique identifier property to extract
product_uid = next((obj for obj in weaviate_properties if obj.lower() == os.environ["PRODUCT_ID"].lower()), weaviate_properties[0])
ids = get_property_value(result, product_uid)

In [31]:
print(ids)

['Organic Superfood Powder', 'Organic Superfood Blend', 'Essential Oil Set', 'Essential Oil Set', 'Blender', 'Protein Powder', 'Fitness Tracker', 'Superfood Smoothie Mix', 'Fitness Tracker Watch', 'Fitness Tracker']


In [32]:
def query_table_single_where(table, filter_col, in_values):
    """Returns records from table where value from in_values is foundin filter_col"""
    in_values = ', '.join(f"'{i}'" for i in ids)
    result = session.sql(f"""
    SELECT * 
    FROM {table}
    WHERE
    {filter_col}
    IN 
    ({in_values})
    limit 10
    """)
    return result.to_pandas()

In [33]:
table_results = query_table_single_where(product_table, product_uid, ids)
table_results.head()

,ORGANIZATION_NAME,PRODUCT_NAME,PRODUCT_CATEGORY,PRODUCT_DESCRIPTION,REWARD,QUALIFICATION,HYPERLINK,REWARD_TYPE,DESCRIPTION
0,Fitness Enthusiasts,Protein Powder,Fitness,Fuel your workouts and support muscle growth w...,$10,Purchase Protein Powder and share a photo of y...,www.fitnessenthusiasts.com/proteinpowder,gift card,Company Fitness Enthusiasts is offering a $10 ...
1,Wellness Enthusiasts,Essential Oil Set,Health & Wellness,Enhance your well-being with our Essential Oil...,$25,Purchase Essential Oil Set and share a photo o...,www.wellnessenthusiasts.com/oilset,gift card,Company Wellness Enthusiasts is offering a $25...
2,Fitness Enthusiasts,Fitness Tracker,Health & Fitness,Track your workouts and monitor your health wi...,$20,Purchase Fitness Tracker and enter code FITEN2...,www.fitnessenthusiasts.com/tracker,discount code,Company Fitness Enthusiasts is offering a $20 ...
3,Wellness Enthusiasts,Essential Oil Set,Health & Wellness,Indulge in the therapeutic benefits of aromath...,$20,Purchase Essential Oil Set and write a review ...,www.wellnessenthusiasts.com/essentialoilset,cash back,Company Wellness Enthusiasts is offering a $20...
4,Fitness Junkies,Fitness Tracker,Sports & Outdoors,Track your fitness goals and stay motivated wi...,$30,Purchase Fitness Tracker and write a review on...,www.fitnessjunkies.com/fitnesstracker,cash back,Company Fitness Junkies is offering a $30 cash...
